## Titanic Data Analysis (Refactored)

This notebook analyzes the Titanic dataset as part of my statistics class final project.  
All code is my own, originally written in 2022 and polished in 2025 for my portfolio.

We will use Pandas, NumPy, and Plotly for data cleaning, feature engineering, and visualization.

In [3]:
# Import necessary libraries
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# Load the Titanic dataset
titanic = pd.read_excel('titanic3.xlsx')

# Display the first few rows to understand the data structure
titanic.head(4)

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,1,0,"Allison, Miss. Helen Loraine",female,2.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1,2,113781,151.5500,C22 C26,S,NaN,135.0,"Montreal, PQ / Chesterville, ON"


### Selecting Relevant Columns

We will focus on the columns most relevant for survival analysis:  
- Passenger class (`pclass`)
- Survival status (`survived`)
- Sex (`sex`)
- Age (`age`)
- Ticket fare (`fare`)
- Lifeboat assignment (`boat`)

In [5]:
# Select relevant columns (creates a view, not a copy)
cols = ['pclass', 'survived', 'sex', 'age', 'fare', 'boat']
titanic_view = titanic[cols]

# Show basic info and missing values
titanic_view.info()
titanic_view.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   pclass    1309 non-null   int64  
 1   survived  1309 non-null   int64  
 2   sex       1309 non-null   object 
 3   age       1046 non-null   float64
 4   fare      1308 non-null   float64
 5   boat      486 non-null    object 
dtypes: float64(2), int64(2), object(2)
memory usage: 61.5+ KB


pclass        0
survived      0
sex           0
age         263
fare          1
boat        823
dtype: int64

### Data Cleaning and Feature Engineering

Now we will:
- Remove rows with missing values in 'age' or 'boat' (since these are needed for our analysis).
- Create new categorical columns for survival/sex and age/sex groups using efficient, vectorized code.

In [6]:
# Remove rows with missing 'age' or 'boat'
titanic_clean = titanic_view.dropna(subset=['age', 'boat']).copy()

# Create a categorical column for survival/sex group
# 0: Female Died, 1: Male Died, 2: Female Survived, 3: Male Survived
titanic_clean['survival_sex_group'] = (
    titanic_clean['survived'].astype(int).astype(str) + "_" + titanic_clean['sex']
)
group_map = {'0_female': 0, '0_male': 1, '1_female': 2, '1_male': 3}
titanic_clean['survival_sex_group'] = titanic_clean['survival_sex_group'].map(group_map)

# Create a categorical column for age/sex group
def age_sex_group(row):
    if row['age'] <= 15 and row['sex'] == 'male':
        return 'boy'
    elif row['age'] > 15 and row['sex'] == 'male':
        return 'man'
    elif row['age'] <= 15 and row['sex'] == 'female':
        return 'girl'
    else:
        return 'woman'

titanic_clean['age_sex_group'] = titanic_clean.apply(age_sex_group, axis=1)

# Show the first few rows to verify
titanic_clean.head()

,pclass,survived,sex,age,fare,boat,survival_sex_group,age_sex_group
0,1,1,female,29.0000,211.3375,2,2,woman
1,1,1,male,0.9167,151.5500,11,3,boy
5,1,1,male,48.0000,26.5500,3,3,man
6,1,1,female,63.0000,77.9583,10,2,woman
8,1,1,female,53.0000,51.4792,D,2,woman


### Lifeboat Order and Fare Binning

We will:
- Assign a numeric order to each lifeboat code for easier analysis.
- Remove ambiguous or outlier boat/fare values.
- Bin fares into categories for grouped analysis.

In [7]:
# Assign numeric order to lifeboat codes
boat_order_map = {
    '7': 1, '5': 2, '3': 3, '8': 4, '1': 5, '6': 6, '16': 7, '14': 8, '12': 9, '9': 10,
    '11': 11, '13': 12, '15': 13, '2': 14, '10': 15, '4': 16, 'C': 17, 'D': 18, 'B': 19, 'A': 20
}
titanic_clean['boat_order'] = titanic_clean['boat'].map(boat_order_map)

# Remove ambiguous boat values and fare outlier
ambiguous_boats = ['5 9', '5 7', '8 10', '13 15 B', 'C D', '13 15']
titanic_clean = titanic_clean[~titanic_clean['boat'].isin(ambiguous_boats)]
titanic_clean = titanic_clean[titanic_clean['fare'] != 512.3292]

# Bin fares into categories of 20 units each
fare_bins = np.arange(0, titanic_clean['fare'].max() + 20, 20)
titanic_clean['fare_bin'] = pd.cut(titanic_clean['fare'], bins=fare_bins, labels=False, include_lowest=True) + 1

# Show the first few rows to verify
titanic_clean[['boat', 'boat_order', 'fare', 'fare_bin']].head()

,boat,boat_order,fare,fare_bin
0,2,14.0,211.3375,11
1,11,11.0,151.5500,8
5,3,3.0,26.5500,2
6,10,15.0,77.9583,4
8,D,18.0,51.4792,3


### Survival by Sex Group

Let’s visualize the number of passengers in each survival/sex group:
- 0: Female Died
- 1: Male Died
- 2: Female Survived
- 3: Male Survived

In [8]:
# Count passengers in each survival/sex group
surv_sex_counts = titanic_clean['survival_sex_group'].value_counts().sort_index()

fig = go.Figure(data=go.Bar(
    x=['0: Female Died', '1: Male Died', '2: Female Survived', '3: Male Survived'],
    y=surv_sex_counts.values,
    marker_color='rgb(55, 83, 109)'
))
fig.add_annotation(text="0: Female Died, 1: Male Died",
                  xref="paper", yref="paper",
                  x=-0.005, y=-0.17, showarrow=False)
fig.add_annotation(text="2: Female Survived, 3: Male Survived",
                  xref="paper", yref="paper",
                  x=-0.005, y=-0.24, showarrow=False)
fig.update_layout(title_text='Survival by Sex Group')
fig.show()

### Survival by Sex Group Table

Here’s a summary table of the counts for each group:

In [9]:
# Create a summary table
import pandas as pd
table_data = {
    'Survival/Sex Group': ['0: Female Died', '1: Male Died', '2: Female Survived', '3: Male Survived'],
    'Count': surv_sex_counts.values
}
summary_df = pd.DataFrame(table_data)
summary_df

,Survival/Sex Group,Count
0,0: Female Died,1
1,1: Male Died,6
2,2: Female Survived,271
3,3: Male Survived,127


### Heatmap: Survival Status vs Passenger Class

This heatmap shows the relationship between passenger class and survival status.

In [10]:
import plotly.express as px

fig = px.density_heatmap(
    titanic_clean,
    x="survival_sex_group",
    y="pclass",
    marginal_x="histogram",
    marginal_y="histogram",
    title="Heatmap: Survival Status vs Passenger Class"
)
fig.add_annotation(
    text="0: Female Died, 1: Male Died, 2: Female Survived, 3: Male Survived",
    xref="paper", yref="paper",
    x=0.15, y=-0.25, showarrow=False
)
fig.show()

---
## Titanic Data Analysis

This notebook is part of the final project for my statistics class in 2022 at Universidad Tecnológica de Pereira, supervised by Beatriz Martinez.

The entirety of this notebook was written by me in 2022 and polished in 2025 for expository purposes in this portfolio.

The goal of this project was to conduct research and find insights based on the course content.

For this notebook, we will use libraries like Pandas, NumPy, and Plotly for their versatility, capabilities, and standardization in the data analysis industry.

In [1]:
import numpy as np
import pandas as pd
import plotly.express as px 
import plotly.graph_objects as go

In [2]:
#Carga del Framework
Titanic=pd.read_excel('titanic3.xlsx')

In [3]:
#implementacion y organizacion de los frameworks necesarios para el estudio.
survived = Titanic["survived"]
Filtered=Titanic[['pclass','survived','sex','age','fare','boat']]#*
Filtered2=Titanic[["survived","sex"]]
Filtered3=Titanic[["survived","pclass"]]

#Creacionde Filterdn/ Base de datos que asigna vairables categorivas para Sexo y edad
Filteredn=Filtered.copy()
NewVar=[0]*len(Filteredn)
n=len(Filteredn)
for i in range(n):
    if Filteredn.iloc[i,1]==0 and Filteredn.iloc[i,2]=='female':
        NewVar[i]=0
    if Filteredn.iloc[i,1]==0 and Filteredn.iloc[i,2]=='male':
        NewVar[i]=1
    if Filteredn.iloc[i,1]==1 and Filteredn.iloc[i,2]=='female':
        NewVar[i]=2
    if Filtered.iloc[i,1]==1 and Filtered.iloc[i,2]=='male':
        NewVar[i]=3
e=pd.Series(NewVar)
Filteredn['Sobrevivientes']=e
#Filteredn.head()


#Limpieza de la categoria 'boat' asignada en el nuevo dataframe
#NEWDF= Nuevo datafrem/ Filteredn = datafreme que contiene el filtrado de la db original
Newdf=Filteredn.copy()
Newdf=Newdf[Newdf['boat'].notna()]
Newdf=Newdf[Newdf['age'].notna()]
#Newdf.head()


#Asignacion de RangoSex para Newdf
p=len(Newdf)
List2=[]
for m in range(len(Newdf)):
    if Newdf.iloc[m,3]<=15.0 and Newdf.iloc[m,2]=='male':
        List2.append('niño')
    if Newdf.iloc[m,3]>15.0 and Newdf.iloc[m,2]=='male':
        List2.append('Hombre')
    if Newdf.iloc[m,3]<=15.0 and Newdf.iloc[m,2]=='female':
        List2.append('niña')
    if Newdf.iloc[m,3]>15.0 and Newdf.iloc[m,2]=='female':
        List2.append('Mujer')
b=pd.Series(List2)
Newdf['RangoSex']=b
#Newdf.head()

oo=[0]*len(Filtered)
i=0
for i in range(len(Filtered)):
    if Filtered.iloc[i,5]=='7':
        oo[i]=1
    if Filtered.iloc[i,5]=='5':
        oo[i]=2
    if Filtered.iloc[i,5]=='3':
        oo[i]=3
    if Filtered.iloc[i,5]=='8':
        oo[i]=4
    if Filtered.iloc[i,5]=='1':
        oo[i]=5
    if Filtered.iloc[i,5]=='6':
        oo[i]=6
    if Filtered.iloc[i,5]=='16':
        oo[i]=7
    if Filtered.iloc[i,5]=='14':
        oo[i]=8
    if Filtered.iloc[i,5]=='12':
        oo[i]=9
    if Filtered.iloc[i,5]=='9':
        oo[i]=10
    if Filtered.iloc[i,5]=='11':
        oo[i]=11
    if Filtered.iloc[i,5]=='13':
        oo[i]=12
    if Filtered.iloc[i,5]=='15':
        oo[i]=13
    if Filtered.iloc[i,5]=='2':
        oo[i]=14
    if Filtered.iloc[i,5]=='10':
        oo[i]=15
    if Filtered.iloc[i,5]=='4':
        oo[i]=16
    if Filtered.iloc[i,5]=='C':
        oo[i]=17
    if Filtered.iloc[i,5]=='D':
        oo[i]=18
    if Filtered.iloc[i,5]=='B':
        oo[i]=19
    if Filtered.iloc[i,5]=='A':
        oo[i]=20
#["A","B","D","C","4","10","2","15","13","11","9","12","14","16","6","1","8","3","5","7"]
listn=pd.Series(oo)
Newdf['Orden']=listn
Newdf.head(100)


#Filtrado para Newdf de orden de barcos y categorizacion de los precios
values=['5 9', '5 7', '8 10','13 15 B', 'C D','13 15']

Newdf=Newdf[Newdf.boat.isin(values)==False]
Newdf=Newdf[Newdf.fare != 512.3292]
CategoryList=[0]*len(Filtered)
for i in range(len(Filtered['pclass'])):
    if Filtered.iloc[i,4]<=20.0:
        CategoryList[i]=1
    if Filtered.iloc[i,4]>20.0 and Filtered.iloc[i,4]<=40:
        CategoryList[i]=2
    if Filtered.iloc[i,4]>40.0 and Filtered.iloc[i,4]<=60.0:
        CategoryList[i]=3
    if Filtered.iloc[i,4]>60.0 and Filtered.iloc[i,4]<=80.0:
        CategoryList[i]=4
    if Filtered.iloc[i,4]>80.0 and Filtered.iloc[i,4]<=100.0:
        CategoryList[i]=5
    if Filtered.iloc[i,4]>100.0 and Filtered.iloc[i,4]<=120.0:
        CategoryList[i]=6
    if Filtered.iloc[i,4]>120.0 and Filtered.iloc[i,4]<=140.0:
        CategoryList[i]=7
    if Filtered.iloc[i,4]>140.0 and Filtered.iloc[i,4]<=160.0:
        CategoryList[i]=8
    if Filtered.iloc[i,4]>160.0 and Filtered.iloc[i,4]<=180:
        CategoryList[i]=9
    if Filtered.iloc[i,4]>180.0 and Filtered.iloc[i,4]<=200.0:
        CategoryList[i]=10
    if Filtered.iloc[i,4]>200.0 and Filtered.iloc[i,4]<=220.0:
        CategoryList[i]=11
    if Filtered.iloc[i,4]>220.0 and Filtered.iloc[i,4]<=240.0:
        CategoryList[i]=12
    if Filtered.iloc[i,4]>240.0 and Filtered.iloc[i,4]<=260.0:
        CategoryList[i]=13
    if Filtered.iloc[i,4]>260.0 and Filtered.iloc[i,4]<=280.0:
        CategoryList[i]=14
    if Filtered.iloc[i,4]>280.0 and Filtered.iloc[i,4]<=300.0:
        CategoryList[i]=15
    if Filtered.iloc[i,4]>300.0 and Filtered.iloc[i,4]<=320.0:
        CategoryList[i]=16
    if Filtered.iloc[i,4]>320.0 and Filtered.iloc[i,4]<=340.0:
        CategoryList[i]=17
    if Filtered.iloc[i,4]>340.0 and Filtered.iloc[i,4]<=360.0:
        CategoryList[i]=18
    if Filtered.iloc[i,4]>360.0 and Filtered.iloc[i,4]<=380.0:
        CategoryList[i]=19
    if Filtered.iloc[i,4]>380.0 and Filtered.iloc[i,4]<=400.0:
        CategoryList[i]=20
otherlist=pd.Series(CategoryList)
Newdf['RangoPrecio']=otherlist


BoatZeros=Filteredn.copy()


### Graficos de Barras


In [4]:
count=Filteredn.groupby(['Sobrevivientes']).count()
fig = go.Figure(data=go.Bar(
    x = ['0', '1', '2' , '3'],
    y = count['pclass'],
    marker_color = 'rgb(55, 83, 109)' #Color de las barras (rgb)
))
fig.add_annotation(text="0. Mujeres Muertas 1. Hombres Muertos",
                  xref="paper", yref="paper",
                  x=-0.005, y=-0.17, showarrow=False)
fig.add_annotation(text="2. Mujeres Vivas     3. Hombres vivos",
                  xref="paper", yref="paper",
                  x=-0.005, y=-0.24, showarrow=False)
fig.update_layout(title_text='Sobrevivivientes vs Sexo')

In [22]:
#Tabla sin organizar para la grafica anterior
Othervar=Filteredn.groupby(['Sobrevivientes']).count().copy()
Othervar.head()

,pclass,survived,sex,age,fare,boat
Sobrevivientes,,,,,,
0,127,127,127,96,127,1
1,682,682,682,523,681,8
2,339,339,339,292,339,318
3,161,161,161,135,161,159


In [24]:
#Tabla de la anterior grafica organizada
table = go.Figure(data=[go.Table(
    header=dict(values=['Sobrevivientes','Conteo'],
                fill_color='black',
                align='left',
                font=dict(color='white', size=12)),
    cells=dict(values=[[0,1,2,3],Othervar.pclass],
               fill_color='lavender',
               align='left'))
])

table.show()

In [ ]:
#Mapas de Calor

In [6]:
fig2 = px.density_heatmap(Filteredn , 
                        x = "Sobrevivientes", 
                        y = "pclass", 
                        marginal_x = "histogram", 
                        marginal_y = "histogram",
                        title = "Mapa de calor")
fig2.add_annotation(text = "Mapa que realciona clase con estado de supervivencia",
                  xref = "paper", yref="paper",
                  x=0.15, y=-0.25, showarrow=False)
fig2.show()

In [23]:
#Esta gráfica no aparece en el informe(Fue descartada), pero gracias a ella se encontró la 
#tendencia del precio del tiquete y que era necesario sacar algunos outlayers
graph = px.density_heatmap(Filteredn , 
                        x="fare", 
                        y="Sobrevivientes", 
                        marginal_x="histogram", 
                        marginal_y="histogram",
                        range_color=[0,200],
                        title="Mapa de calor")
graph.add_annotation(text="Mapa que realciona precio del tiquete con estado de supervivencia",
                  xref="paper", yref="paper",
                  x=-0.035, y=1.15, showarrow=False)
graph.add_annotation(text="0. Mujeres Muertas 1. Hombres Muertos",
                  xref="paper", yref="paper",
                  x=-0.005, y=-0.17, showarrow=False)
graph.add_annotation(text="2. Mujeres Vivas     3. Hombres vivos",
                  xref="paper", yref="paper",
                  x=-0.005, y=-0.24, showarrow=False)


graph.show()

In [8]:
""""
A partir de este grafico se encontro la poporcion de hombre y mujeres segun si eran niños o mayores
y si habien sobrevivido
"""""
graph2 = px.density_heatmap(Newdf , 
                        x="Orden", 
                        y="RangoSex", 
                        marginal_x="histogram", 
                        marginal_y="histogram",
                        range_color=[0,60],
                        title="Mapa de calor")
graph2.show()

In [ ]:
#Graficos de dispersion

In [9]:

disp = px.strip(Filteredn, x="fare", y="Sobrevivientes", orientation="h", color="pclass")
disp.update_layout(legend_title_text='Clase')
disp.update_xaxes(title_text='Precio del tiquete')
disp.update_yaxes(title_text='Estado de superviventcia')
disp.add_annotation(text="0. Mujeres Muertas 1. Hombres Muertos",
                  xref="paper", yref="paper",
                  x=-0.005, y=-0.17, showarrow=False)
disp.add_annotation(text="2. Mujeres Vivas     3. Hombres vivos",
                  xref="paper", yref="paper",
                  x=-0.005, y=-0.24, showarrow=False)
disp.show()

In [10]:
# Con este conjunto de datos se obtuvo la mejor correalcion de pearson}

disp1 = px.strip(Newdf, x="pclass", y="Orden", orientation="h",color='pclass')
disp1.update_layout(legend_title_text='Clase')
disp1.update_xaxes(title_text='Clase')
disp1.update_yaxes(title_text='Bote salvavidas')
disp1.add_annotation(text="CP=0.4782",
                  xref="paper", yref="paper",
                  x=-0.005, y=-0.17, showarrow=False)

disp1.show()

In [11]:



disp2 = px.scatter(Newdf, x="fare", y="boat", orientation="h",marginal_x="histogram",marginal_y="rug",
                category_orders={"boat":["A","B","D","C","4","10","2","15","13","11","9","12","14","16","6","1","8","3","5","7"]})
disp2.update_layout(legend_title_text='Clase')
disp2.update_xaxes(title_text='Precio del tiquete')
disp2.update_yaxes(title_text='Botes')

disp2.show()

In [12]:


disp1 = px.strip(Newdf, x="age", y="Orden", orientation="h",color='pclass',
                category_orders={"boat":["A","B","D","C","4","10","2","15","13","11","9","12","14","16","6","1","8","3","5","7"]})
disp1.update_layout(legend_title_text='Clase')
disp1.update_xaxes(title_text='Edad')
disp1.update_yaxes(title_text='Bote salvavidas')
disp1.add_annotation(text="CP=-0.2198",
                  xref="paper", yref="paper",
                  x=-0.005, y=-0.17, showarrow=False)

disp1.show()

In [37]:
#Media Muestral (Muestra Grande) Precio que se pagó por el tiquete
Newdf['fare'].mean()

50.26672938271604

In [18]:
#Media muestral(Muestra pequeña) Clase perteneciente dentro del barco
Newdf[['pclass','fare']].mean()

pclass     1.874074
fare      50.266729
dtype: float64

In [19]:
#Varianza Muestral (Muestra pequeña de datos) Clase ocupada dentro del barco
Newdf['pclass'].var()

0.7192519251925192

In [21]:
#VArianza Muestral (muestra grande de datos) Pecio que se pagó por los tiquetes
Newdf['fare'].var()

3374.9707615557923

In [ ]:
#Funcion de distrubución


In [44]:
#Media Muestral de la muestra de precios categorizada en subconjuntos de 20 en 20
Newdf['RangoPrecio'].mean()

2.9901234567901236

In [45]:
#Varianza muestral de el precio categorizado en conjuntos de 20 en 20
Newdf['RangoPrecio'].mean()

2.9901234567901236

In [46]:
#Funcion de distribucion. 
""""
Despues de encontrar que el precio de los tiquetes tenia una tendencia exponenecia, se agrupó este en categorías,
se halló l amedia y se trazó una fmp dividiendo cada barra por el número de datos totales. Despues con la media 
se halló lambda y se trazó la funcion de de distribucion exponencial.
"""""
Lamb=1/2.99 # 
t = np.linspace(0, 15, 100)
t1=[*range(1,14,1)]
Prob=[157,102,38,33,23,12,10,5,3,0,6,4,2,10]
Prob2=np.divide(Prob,405)
fig = px.line(x=t, y=np.exp(-Lamb*t)*Lamb, labels=dict(x='Precio del tiquete',y='P(X=x)',color='-'),
            color=px.Constant("f(x)"))
fig.add_bar(x=t1,y=Prob2, name='fmp')
fig.add_annotation(text="f(x)=(100/299)*e^-((100/299)*x)",
                  xref="paper", yref="paper",
                  x=-0.005, y=-0.17, showarrow=False)

fig.show()

In [ ]:
#Tablas
#Las tablas muestran los primeros 5 datos del dataframe. si se desean mas datos, se pone el numero
#de datos deseados en el argumento de la funcion .head()

In [65]:
#Tabla Filtered
#La tabla Filtered es una tabla inicial con la cuals se escogieron las categorias de interes
table1 = go.Figure(data=[go.Table(
    header=dict(values=list(Filtered.columns),
                fill_color='black',
                align='left',
                font=dict(color='white', size=12)),
    cells=dict(values=[Filtered.pclass,Filtered.survived,Filtered.sex,Filtered.age,Filtered.fare,
                Filtered.boat],
               fill_color='lavender',
               align='left'))
])

table1.show()

In [64]:
#Tabla Filteredn
""""
La tabla Filteredn, es una tabla con la cual se asignaron variables aleatorias para 
hombres y mujeres vivos o muertos y se sacaron valores de ceros en algunos datos
"""""
table2 = go.Figure(data=[go.Table(
    header=dict(values=list(Filteredn.columns),
                fill_color='black',
                align='left',
                font=dict(color='white', size=12)),
    cells=dict(values=[Filteredn.pclass,Filteredn.survived,Filteredn.sex,Filteredn.age,Filteredn.fare,
                Filteredn.boat,Filteredn.Sobrevivientes],
               fill_color='lavender',
               align='left'))
])

table2.show()

In [62]:
#Tabla Newdf
""""
La Tabla Newdf, es una tabla donde se añalizó unicamente la gente que logró entrar a un bote, se sacaron outlayers,
Valores NaN, se agregaron nuevas categorias de orde se salida de los botes y se agrupó el precio de los tiqueten en grupos
de 20 asignando a acada uno una variable aleatoria.
"""""
table3 = go.Figure(data=[go.Table(
    header=dict(values=list(Newdf.columns),
                fill_color='black',
                align='left',
                font=dict(color='white', size=12)),
    cells=dict(values=[Newdf.pclass,Newdf.survived,Newdf.sex,Newdf.age,Newdf.fare,
                Newdf.boat,Newdf.Sobrevivientes,Newdf.RangoSex,Newdf.Orden,Newdf.RangoPrecio],
               fill_color='lavender',
               align='left'))
])

table3.show()